In [1]:
import pdfplumber
import os
import re
import pandas as pd
import numpy as np

path = "data/Key_Stats/"
files = os.listdir(path)


In [2]:
def read_pages(dir):
    with pdfplumber.open(dir) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

def full_text(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text) # only english
    text = re.sub(r'[\n\t]+', ' ', text) # delete '\n' and '\t'
    text = re.sub(r' +', ' ', text) # delete multiple spaces
    return text

def file_id(file):
    pattern = r'[A-Z]{3,3}[0-9]{3,3}'
    return re.findall(pattern, file)

In [30]:
i = 808
doc_dir = path + files[i]
raw = read_pages(doc_dir)
text = full_text(raw).lower()
ID = file_id(files[i])

fee_loc = text.find('what are the fees and charges?')
text_fee = text[fee_loc:]

# match 'nil' or 'no'


In [31]:
pattern1 = r'[0-9]*\.[0-9]+%'
pattern2 = r'[0-9]+%'
pattern3 = r'nil|no|n\/a'
fee_list = ['subscription', 'switching', 'redemption', 'management', 'trustee', 'performance', 'admin']
fee_table = pd.DataFrame(columns = ['id'] + fee_list)

for fee in fee_list:
    loc = text_fee.find(fee)
    if loc == -1:
        print('no ' + fee)
    else:
        fee1 = re.findall(pattern1,text_fee[loc:])
        fee2 = re.findall(pattern2,text_fee[loc:])
        fee3 = re.findall(pattern3,text_fee[loc:])
        
        loc1 = text_fee[loc:].find(fee1[0]) if len(fee1) > 0 else 6000
        loc2 = text_fee[loc:].find(fee2[0]) if len(fee2) > 0 else 6000
        loc3 = text_fee[loc:].find(fee3[0]) if len(fee3) > 0 else 6000
        if loc3 < loc2 and loc3 < loc1:
            print('no' + fee + 'fee') 
        else:
            if loc1 < loc2:
                print(fee + ' fee = ' + fee1[0])
            else:
                print(fee + ' fee = ' + fee2[0])

subscription fee = 5%
switching fee = 0%
redemption fee = 6%


IndexError: list index out of range

In [32]:
re.findall(pattern1,text_fee[loc:])
re.findall(pattern2,text_fee[loc:])
#re.findall(pattern3,text_fee[loc:])

[]

In [44]:
fee_table = pd.read_csv('data/fee_table.csv')

fee_fund_temp = pd.DataFrame(columns = ['id'] + fee_list)
fee_fund_temp['id'] = ID
fee_fund_temp['subscription'] = '5%'
fee_fund_temp['switching'] = np.nan
fee_fund_temp['redemption'] = '6%'
fee_fund_temp['management'] = '1%'
fee_fund_temp['trustee'] = '0.125%'
fee_fund_temp['performance'] = np.nan
fee_fund_temp['admin'] = '0.035%'

fee_table = fee_table.append(fee_fund_temp, ignore_index = True)
fee_table.to_csv('data/fee_table.csv', index = False)

C:\Users\Yan Jin\AppData\Local\Temp\ipykernel_25256\1406032190.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fee_table = fee_table.append(fee_fund_temp, ignore_index = True)


In [45]:
pattern1 = r'[0-9]*\.[0-9]+%'
pattern2 = r'[0-9]+%'
pattern3 = r"nil|no|n\/a"
fee_list = ['subscription', 'switching', 'redemption', 'management', 'trustee', 'performance', 'admin']
#fee_table = pd.DataFrame(columns = ['id'] + fee_list)


for i in range(809, len(files)):
#for i in range(10):
    doc_dir = path + files[i]
    raw = read_pages(doc_dir)
    text = full_text(raw).lower()
    ID = file_id(files[i])
    
    fee_loc = text.find('what are the fees and charges?')
    text_fee = text[fee_loc:]
    #fee_list = ['subscription', 'switching', 'redemption', 'investment management', 'trustee', 'performance', 'admin']
    #fee_table = pd.DataFrame(columns = ['id'] + fee_list)
    fee_fund = pd.DataFrame(columns = ['id'] + fee_list)
    fee_fund['id'] = ID
    for fee in fee_list:
        loc = text_fee.find(fee)
        if loc == -1:
            fee_fund[fee] = np.nan
        else:
            fee1 = re.findall(pattern1,text_fee[loc:])
            fee2 = re.findall(pattern2,text_fee[loc:])
            fee3 = re.findall(pattern3,text_fee[loc:])

            loc1 = text_fee[loc:].find(fee1[0]) if len(fee1) > 0 else 6000
            loc2 = text_fee[loc:].find(fee2[0]) if len(fee2) > 0 else 6000
            loc3 = text_fee[loc:].find(fee3[0]) if len(fee3) > 0 else 6000
            if loc3 < loc2 and loc3 < loc1:
                fee_fund[fee] = np.nan
            else:
                if loc1 < loc2:
                    fee_fund[fee] = fee1[0]
                else:
                    fee_fund[fee] = fee2[0]
    fee_table = pd.concat([fee_table, fee_fund], axis = 0)

In [13]:
re.findall(pattern3, text_fee[loc:])

['n/a', 'no', 'no', 'no', 'n/a', 'n/a', 'n/a', 'no', 'no', 'no', 'no', 'no']

In [46]:
fee_table.to_csv('data/fee_table.csv', index = False)